In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import scipy
import torch
import torch.nn as nn
import os
import time

from random import shuffle
from scipy.stats import gamma, beta, nbinom, poisson
from scipy.special import gammaln, betaln
from torch.autograd import Variable
import sys 
sys.path.append('../../')
from utils import *
from mdns import ClassificationSingleLayerMDN, Trainer

In [ ]:
%matplotlib inline

mpl.rcParams['axes.titlesize'] = 20
mpl.rcParams['axes.labelsize'] = 17
mpl.rcParams['ytick.labelsize'] = 12
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['legend.fontsize'] = 16
mpl.rcParams['figure.figsize'] = (15, 5)

## Generate Poisson data, compare Poisson vs. negBin model

I will train a MDN to approximate the posterior over model indices. As models I use a Poisson with Gamma prior and a negative Binomial sampled from a Poisson-Gamma mixture with Gamma priors on the Gamma shape and scale parameters. 

As training data I will generate a large data set containing samples from both models generated with the corresponding priors. The MDN gets the mean and variance of the data set as input and outputs a probability vector over models. 

### Controlling difficulty via over / under dispersion 
The prior parameters of the conjugate Gamma prior of the Poisson model and the two Gamma priors of the NB model are chosen such that on average the data sets generated by the models have idenitcal sample means. The difficulty therefore arises from the amount of overdispersion in the NB model: The model comparison problem becomes easier to solve as the variance in the samples form the NB model is larger than the mean. 

In [ ]:
sample_size = 100
n_samples = 10000

# set RNG
seed = 2
np.random.seed(seed)

time_stamp = time.strftime('%Y%m%d%H%M_')
figure_folder = '../figures/'

# set prior parameters 
# set the shape or scale of the Gamma prior for the Poisson model
k1 = 2.0 
# set the shape and scale of the prior on the shape of the Gamma for the mixture to be broad 
theta2 = 2.0
k2 = 5
# set the shape and scale of the prior on the scale of the Gamma for the mixture to be small 
# this will make the variance and could be the tuning point of the amount of overdispersion / difficulty
theta3 = .1
k3 = 20

# then the scale of the Gamma prior for the Poisson is given by 
theta1 = (k2 * theta2 * k3 * theta3) / k1
print(theta1, k1)

# set the priors 
prior_lam = scipy.stats.gamma(a=k1, scale=theta1)
prior_k = scipy.stats.gamma(a=k2, scale=theta2)
prior_theta = scipy.stats.gamma(a=k3, scale=theta3)

## Visualize the priors

In [ ]:
values = np.linspace(0, 30, 1000)
ps = np.linspace(1e-8, 1 - 1e-8, 1000)
plt.figure(figsize=(15, 10))

p_pdf = lambda p: np.power(1-p, -2) * gamma.pdf(p / (1-p), a=k3, scale=theta3)

plt.subplot(311)
plt.plot(values, gamma.pdf(values, a=k1, scale=theta1), label=r'Prior on Poisson $\lambda$')
plt.legend()

plt.subplot(312)
plt.plot(values, gamma.pdf(values, a=k2, scale=theta2), label=r'Prior on NB $r$')
plt.legend()

plt.subplot(313)
plt.plot(ps, p_pdf(ps), label=r'Prior on NB $p$')
plt.legend();

## Generate a large data set of triplets (m, theta, sx)

Then separate it into sets for model 1 and model 2 and train the phi networks separately. 

In [ ]:
# generate one half poisson data    
lambs, X1 = sample_poisson(prior_lam, int(n_samples / 2), sample_size)

# generaze second half NB data using the lambdas for Poisson to get thetas for NB 
thetas, X2 = sample_poisson_gamma_mixture_matched_means(prior_k, lambs, int(n_samples / 2), sample_size)

In [ ]:
X = np.vstack((X1, X2))
m = np.hstack((np.zeros(X1.shape[0]), np.ones(X2.shape[0]))).squeeze().astype(int)

In [ ]:
# get shuffled indices 
shuffle_indices = np.arange(n_samples)
np.random.shuffle(shuffle_indices)

# shuffle the data 
X = X[shuffle_indices, ]
m = m[shuffle_indices].tolist()

# calculate summary stats 
X = np.vstack((X.mean(axis=1), X.var(axis=1))).T

# normalize 
X, norm = normalize(X)

## Define network for fitting the model posterior, optimizer and trainer

In [ ]:
# define a MoG model with n_params + 1 inputs: data dimensions plus model index 
model = ClassificationSingleLayerMDN(ndim_input=2, n_hidden=10)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

trainer = Trainer(model, optimizer, verbose=True, classification=True)

In [ ]:
# train
loss_trace = trainer.train(X, m, n_epochs=200, n_minibatch=100)

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(loss_trace)
plt.xlabel('iterations')
plt.ylabel('loss')
plt.title('Loss');

## Visualize input-output function of the network

In [ ]:
x, y = np.meshgrid(np.linspace(0, 40, 100), np.linspace(0, 40, 100))

In [ ]:
ppoi_mat = np.zeros((100, 100))
softmax = nn.Softmax(dim=0)

for i in range(x.shape[0]): 
    for j in range(x.shape[0]): 
        stats_o, norm = normalize(np.array([x[i, j], y[i, j]]), norm)
        (out_act) = model(Variable(torch.Tensor(stats_o)))
    
        # in this vector, index 0 is Poi, index 1 is NB
        posterior_probs = softmax(out_act).data.numpy()
        ppoi_mat[i, j] = posterior_probs[0]

In [ ]:
plt.imshow(ppoi_mat, origin='lower', extent=[x.min(), x.max(), y.min(), y.max()], aspect='auto')
plt.xlabel('Mean')
plt.ylabel('Variance')
plt.colorbar(label='P(Poisson)');
plt.tight_layout()
save_figure(filename='network_visualization_2d_M{}N{}'.format(sample_size, n_samples), 
            folder=figure_folder, 
            time_stamp=time_stamp)

## Make predictions with test data 

In [ ]:
# generate test data from both models
testX = []
testSX = []
thetas = []
m = []
n_test_samples = 20
sample_size = sample_size
markers = []

for sample_idx in range(n_test_samples): 
    
    # sample model index 
    m.append(int(np.round(np.random.rand())))
    
    if m[sample_idx] == 0: 
        # sample poisson 
        theta, x = sample_poisson(prior_lam, 1, sample_size)
        markers.append('x')
    if m[sample_idx] == 1: 
        # sample poisson 
        theta, x= sample_poisson_gamma_mixture(prior_k, prior_theta, 1, sample_size)
        markers.append('o')

    # calculate mean and var as summary stats 
    testX.append(x)
    testSX.append([np.mean(x), np.var(x)])
    thetas.append(theta)
    
testX = np.array(testX).squeeze()
testSX = np.array(testSX)
testm = m

In [ ]:
Xtest, testm = generate_poisson_nb_data_set(n_test_samples, sample_size, prior_lam, prior_k, prior_theta)
testSX = calculate_stats_toy_examples(Xtest)

In [ ]:
test_ppoi = []
for xtest in testSX: 
    stats_o, norm = normalize(xtest, norm)        
    (out_act) = model(Variable(torch.Tensor(stats_o)))

    # in this vector, index 0 is Poi, index 1 is NB
    test_ppoi.append(softmax(out_act).data.numpy()[0])    

test_ppoi = np.array(test_ppoi)
test_ppoi[test_ppoi == 1.] = 0.999
test_ppoi[test_ppoi < 1e-12] = 1e-12
lbf_pred = np.log(test_ppoi) - np.log(1 - test_ppoi)

In [ ]:
plt.plot(test_ppoi, 'o-')
testm

In [ ]:
poi_mask = np.array(testm) == 0
nb_mask = np.logical_not(poi_mask)
x, y = np.meshgrid(np.linspace(0, 100, 100), np.linspace(0, 100, 100))

# color norm 
colornorm = mpl.colors.Normalize(vmin=test_ppoi.min(), vmax=test_ppoi.max())

plt.scatter(x=testSX[poi_mask, 0], y=testSX[poi_mask, 1], 
            cmap='viridis', c=test_ppoi[poi_mask], marker='x', 
           label='Poisson', norm=colornorm)
plt.scatter(x=testSX[nb_mask, 0], y=testSX[nb_mask, 1], 
            cmap='viridis', c=test_ppoi[nb_mask], marker='^', label='NegBin', norm=colornorm)

plt.colorbar(label='p(poisson)', orientation='vertical')
plt.xlabel('mean')
plt.ylabel('variance')
#plt.contour(x, y, ppoi_mat)
plt.yscale('log')
plt.xscale('log')
plt.legend();

## Compare network predictions to analytical posterior probabilities

The network outputs a vector of posterior model probabilities. Ideally, we have the same values in analytical form. Up to now we just derived the analytical form of the individual model evidences and from that the analytical Bayes factor. This we can compare to the predicted posterior probability because we have a uniform model prior. With a uniform model prior $(p(M_1) = p(M_2))$ the posterior ratio is the same as the evidence ration (Bayes factor):

\begin{align}
BayesFactor = \frac{p(D | M_1)}{p(D | M_2)} = \frac{p(M_1 | D) p(M_1)}{p(M_2 | D) p(M_2)} = \frac{p(M_1 | D)}{p(M_2 | D)}
\end{align}

Thus, we can just calculate the individual model evidences and compare their ratio to the ratio of the output of the network. 

Alternatively, we can derive the analytical posterior probabilities for each model and compare it directly to the probabilities predicted by the network. 

\begin{align}
p(M_i | D) &= \frac{p(D|M_i)p(M_i)}{p(D)} \\
&= \frac{p(D|M_i)p(M_i)}{\sum_j^{|M|} p(D | M_j)} \\ 
&= \frac{\int p(D|\theta)p(\theta|M_i)d\theta \; p(M_i)}{\sum_j^{|M|} \int p(D|\theta)p(\theta|M_j)d\theta}
\end{align}

In [ ]:
# set up a grid of values around the priors 
# take grid over the whole range of the priors
k_start = scipy.stats.gamma.ppf(1e-8, a=k2)
k_end = scipy.stats.gamma.ppf(1 - 1e-8, a=k2)

theta_start = scipy.stats.gamma.ppf(1e-8, a=k3)
theta_end = scipy.stats.gamma.ppf(1 - 1e-8, a=k3)

In [ ]:
# calculate nb evidence for every test set 
nb_evidences = []
poi_evidences = []

for ii, (x, mi) in enumerate(zip(Xtest, testm)):
    (lquad, err) = scipy.integrate.dblquad(func=nb_evidence_integrant_direct,
                                           a=theta_start / (1 + theta_start),
                                           b=theta_end / (1 + theta_end),
                                           gfun=lambda x: k_start, hfun=lambda x: k_end, 
                                           args=[x, prior_k, prior_theta])
    print(mi, 'NB', np.log(lquad))
    nb_evidences.append(np.log(lquad))

    le = poisson_evidence(x, k=k1, theta=theta1, log=True)
    print(mi, 'Poi', le)
    poi_evidences.append(le)

In [ ]:
poi_evidences = np.array(poi_evidences)
nb_evidences = np.array(nb_evidences)
lbf_ana = poi_evidences - nb_evidences
ppoi_ana = calculate_pprob_from_evidences(np.exp(poi_evidences), np.exp(nb_evidences))

In [ ]:
plt.plot(lbf_pred, 'o-', label='predicted')
plt.plot(lbf_ana, 'o-', label='analytical')
plt.xlabel('test data set index')
plt.ylabel('log bayes factor')
plt.legend();
plt.tight_layout()
save_figure(filename='bayesfactor_prediction_evaluation_M{}N{}'.format(sample_size, n_samples), 
            folder=figure_folder, 
            time_stamp=time_stamp)
plt.grid()

In [ ]:
plt.plot(test_ppoi, 'o-', label='predicted')
plt.plot(ppoi_ana, 'o-', label='analytical')
plt.xlabel('test data set index')
plt.ylabel(r'$Pr(M_{poi} | X)$')
plt.legend();
plt.tight_layout()
save_figure(filename='posterior_prediction_evaluation_M{}N{}'.format(sample_size, n_samples), 
            folder=figure_folder, 
            time_stamp=time_stamp)